In [41]:
import requests
import pandas as pd
import defs
from datetime import datetime, timedelta
import pytz
import defs

Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 5000 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 2.016 llamadas de la API por cada toda una semana.  
**NO HAY CONVERSION DE HORAS, LA HORA SOLO ES UNA Y VAMOS A USAR FORMATO Z UTC+0**  
- FORMATO RFC 3339:
    - 2024-07-03T03:08:00.000000000Z  (le resto 4 horas al UTC+0)


Como el API usa los valores Z UTC+0, voy a basar toda mi logica en este formato, bajo esa premisa mis horas importantes son:

- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z

*NOTA:*  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [170]:
#Funcion para obtener hora actual de NY en formato Z UTC-0
import pytz
from datetime import datetime

def obtener_hora_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Formatear la fecha y hora en el formato deseado
    hora_formateada = ahora_ny.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_formateada

# Ejemplo de uso
hora_actual_ny = obtener_hora_nueva_york_rfc3339()
print("Hora actual en Nueva York:", hora_actual_ny)


Hora actual en Nueva York: 2024-07-02T23:14:32.292526Z


Voy a pedile a a API que me entregue las velas los ultimos 7 dias siguiendo la hora de ny que es -4 . Yo manualmente le pongo la hora de inicio para simplificar este paso, luego tendra que ser automatizado.

In [164]:
now_time = obtener_hora_nueva_york_rfc3339()

def restar_siete_dias(fecha_hora_rfc3339):
    # Convertir la cadena RFC3339 a un objeto datetime
    fecha_hora = datetime.strptime(fecha_hora_rfc3339, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Restar 7 días a la fecha y hora
    fecha_hora_menos_siete_dias = fecha_hora - timedelta(days=7)
    
    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    nueva_fecha_hora_rfc3339 = fecha_hora_menos_siete_dias.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return nueva_fecha_hora_rfc3339

restar_siete_dias(now_time)


'2024-06-25T10:19:45.019576Z'

In [67]:
# Obtener las 04:00 hras de la hora del dia hace 7 dias

def obtener_hora_04_00(fecha_hora_rfc3339):
    # Convertir la cadena RFC3339 a un objeto datetime
    fecha_hora = datetime.strptime(fecha_hora_rfc3339, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día
    fecha_04_00 = fecha_hora.replace(hour=4, minute=0, second=0, microsecond=0)
    
    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    nueva_fecha_hora_rfc3339 = fecha_04_00.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    
    return nueva_fecha_hora_rfc3339

# Ejemplo de uso
fecha_hora_original = "2024-06-21T01:28:14.265493Z"
fecha_hora_04_00 = obtener_hora_04_00(fecha_hora_original)
print("Fecha y hora original:", fecha_hora_original)
print("Fecha y hora a las 04:00:", fecha_hora_04_00)



Fecha y hora original: 2024-06-21T01:28:14.265493Z
Fecha y hora a las 04:00: 2024-06-21T04:00:00.000000Z


In [134]:
# Intento de obtener la fecha de inicio en una funcion

def get_7days_ago_date():
    hora_actual_ny = obtener_hora_nueva_york_rfc3339()
    hace_7_dias = restar_siete_dias(hora_actual_ny)
    hace_7_dias_4hrs = obtener_hora_04_00(hace_7_dias)

    return hace_7_dias_4hrs #esta es mi hora de inicio

get_7days_ago_date()

'2024-06-25T04:00:00.000000Z'

Llamada al API y conversion a Dataframe

In [128]:
def get_candles (instrument, granularity, starting_date, count):
    session = requests.Session()
    url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
    params = {
        'granularity' : granularity,
        'price' : "MBA",
        'from' : starting_date,
        'count': count
    }
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    candle_data = response.json()
    return candle_data


def convert_to_df(candle_data):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    our_data = []

    for candle in candle_data["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame(our_data)



In [157]:
# PRUEBO LA FUNCION
instrument = "EUR_USD"
granularity = "M1"
starting_date = hace_7_dias_4hrs

gg = get_candles(instrument, granularity, starting_date, 5000)
convert_to_df(gg)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-23T21:04:00.000000000Z,1,1.06891,1.06891,1.06891,1.06891,1.06861,1.06861,1.06861,1.06861,1.06921,1.06921,1.06921,1.06921
1,2024-06-23T21:05:00.000000000Z,10,1.06898,1.06904,1.06896,1.06900,1.06869,1.06881,1.06869,1.06875,1.06926,1.06930,1.06921,1.06926
2,2024-06-23T21:06:00.000000000Z,12,1.06902,1.06911,1.06901,1.06906,1.06878,1.06890,1.06877,1.06881,1.06926,1.06932,1.06925,1.06932
3,2024-06-23T21:07:00.000000000Z,26,1.06904,1.06912,1.06903,1.06904,1.06878,1.06891,1.06878,1.06884,1.06931,1.06935,1.06924,1.06925
4,2024-06-23T21:08:00.000000000Z,19,1.06909,1.06911,1.06902,1.06902,1.06888,1.06890,1.06878,1.06881,1.06930,1.06932,1.06923,1.06924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-06-27T09:35:00.000000000Z,40,1.06950,1.06959,1.06950,1.06958,1.06943,1.06952,1.06942,1.06951,1.06957,1.06966,1.06957,1.06966
4996,2024-06-27T09:36:00.000000000Z,55,1.06959,1.06963,1.06954,1.06956,1.06952,1.06956,1.06947,1.06949,1.06966,1.06970,1.06961,1.06963
4997,2024-06-27T09:37:00.000000000Z,18,1.06955,1.06956,1.06950,1.06952,1.06948,1.06949,1.06943,1.06945,1.06962,1.06963,1.06957,1.06960
4998,2024-06-27T09:38:00.000000000Z,55,1.06954,1.06966,1.06943,1.06965,1.06946,1.06958,1.06936,1.06958,1.06962,1.06973,1.06950,1.06972


### Logica de Iteracion para obtencion de datos consecutiva del API

In [158]:
#Funcion para obtener la ultima vela del df anterior
def last_candle(candles_df):
    last_candle = candles_df.tail(1)['time']
    return last_candle


def last_candle2(candles_df):
    valor = candles_df.iloc[-1, 0]
    print(valor)
    print(type(valor))
    return valor

last_candle2(gg)

AttributeError: 'dict' object has no attribute 'iloc'

In [161]:
# Intento hacer la llamada al API dentro de otra funcion que llame desde una semana atras 

starting_date = get_7days_ago_date()
instrument = "EUR_USD"
granularity = "M1"

def get_historical_candles(starting_date, instrument, granularity, count=5000):
    today_date = obtener_hora_nueva_york_rfc3339()
    json_data = get_candles(instrument, granularity, starting_date, 5000)
    df = convert_to_df(json_data)
    last_previous_candle = last_candle(df)
    # while last_previous_candle!= today_date:
    #     starting_date = last_previous_candle['time']
    #     json_data = get_candles(instrument, granularity, starting_date, 5000)
    #     df = df.append(convert_to_df(json_data))
    #     last_previous_candle = last_candle(df)
    last_candle = df.iloc[-1, 0]
    if 
    

    return df

get_historical_candles(starting_date, instrument, granularity)
    

<class 'str'>


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-25T04:01:00.000000000Z,37,1.07383,1.07387,1.07383,1.07386,1.07376,1.07380,1.07376,1.07379,1.07390,1.07394,1.07390,1.07394
1,2024-06-25T04:02:00.000000000Z,17,1.07387,1.07387,1.07383,1.07384,1.07380,1.07380,1.07375,1.07376,1.07394,1.07395,1.07391,1.07391
2,2024-06-25T04:03:00.000000000Z,4,1.07382,1.07383,1.07380,1.07380,1.07375,1.07376,1.07373,1.07373,1.07389,1.07390,1.07388,1.07388
3,2024-06-25T04:04:00.000000000Z,15,1.07380,1.07381,1.07380,1.07380,1.07372,1.07374,1.07372,1.07372,1.07388,1.07388,1.07387,1.07387
4,2024-06-25T04:05:00.000000000Z,24,1.07380,1.07380,1.07372,1.07376,1.07373,1.07373,1.07365,1.07368,1.07387,1.07387,1.07380,1.07383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2024-06-28T16:19:00.000000000Z,41,1.07162,1.07169,1.07162,1.07167,1.07155,1.07162,1.07155,1.07160,1.07170,1.07176,1.07170,1.07174
4996,2024-06-28T16:20:00.000000000Z,49,1.07168,1.07173,1.07165,1.07172,1.07160,1.07166,1.07158,1.07165,1.07175,1.07180,1.07172,1.07180
4997,2024-06-28T16:21:00.000000000Z,31,1.07173,1.07186,1.07166,1.07186,1.07166,1.07179,1.07159,1.07179,1.07180,1.07194,1.07174,1.07194
4998,2024-06-28T16:22:00.000000000Z,53,1.07187,1.07196,1.07182,1.07187,1.07180,1.07189,1.07175,1.07180,1.07194,1.07204,1.07189,1.07194


In [162]:
obtener_hora_nueva_york_rfc3339()

'2024-07-02T10:18:26.779180Z'

# UTILIDADES A NECESITAR LUEGO

In [40]:
#Como eliminar el duplicado, hacer luego de juntar los dataframes
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Nombre': ['Ana', 'Luis', 'Ana', 'Carlos', 'Luis'],
        'Edad': [23, 34, 23, 45, 34],
        'Ciudad': ['Madrid', 'Barcelona', 'Madrid', 'Valencia', 'Barcelona']}

df = pd.DataFrame(data)

# Mostrar el DataFrame original
print("DataFrame original:")
print(df)

# Eliminar filas duplicadas
df_sin_duplicados = df.drop_duplicates()

# Mostrar el DataFrame sin duplicados
print("\nDataFrame sin duplicados:")
print(df_sin_duplicados)

DataFrame original:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
2     Ana    23     Madrid
3  Carlos    45   Valencia
4    Luis    34  Barcelona

DataFrame sin duplicados:
   Nombre  Edad     Ciudad
0     Ana    23     Madrid
1    Luis    34  Barcelona
3  Carlos    45   Valencia


In [171]:
import pytz
from datetime import datetime

def obtener_hora_actual_nueva_york_rfc3339():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Convertir la fecha y hora a UTC+0
    ahora_utc = ahora_ny.astimezone(pytz.utc)

    # Formatear la fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = ahora_utc.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_actual_ny_rfc3339 = obtener_hora_actual_nueva_york_rfc3339()
print("Hora actual en Nueva York (RFC3339 UTC+0):", hora_actual_ny_rfc3339)


Hora actual en Nueva York (RFC3339 UTC+0): 2024-07-03T03:14:38.568484Z
